# Projet IA -- clustering

In [1]:
from __future__ import division
import matplotlib.pyplot as plt
import pandas as pds
import seaborn as sns
import numpy as np
import random
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy import stats
%matplotlib inline
np.random.seed(10)
random.seed(10)

## Data import

In [5]:
df = pds.read_csv('./Data/Data.txt', sep="\t") # load a pandas dataframe from csv

df.head()
#df.tail()

,Time,Time Code,Country Name,Country Code,"Annual freshwater withdrawals, total (billion cubic meters) [ER.H2O.FWTL.K3]","Children out of school, primary [SE.PRM.UNER]",CO2 emissions (kt) [EN.ATM.CO2E.KT],"Electricity production from oil, gas and coal sources (% of total) [EG.ELC.FOSL.ZS]","Electricity production from renewable sources, excluding hydroelectric (% of total) [EG.ELC.RNWX.ZS]","Life expectancy at birth, total (years) [SP.DYN.LE00.IN]","PM2.5 air pollution, mean annual exposure (micrograms per cubic meter) [EN.ATM.PM25.MC.M3]","Population, total [SP.POP.TOTL]",Rural population (% of total population) [SP.RUR.TOTL.ZS],Surface area (sq. km) [AG.SRF.TOTL.K2],Total greenhouse gas emissions (kt of CO2 equivalent) [EN.ATM.GHGT.KT.CE],Urban population [SP.URB.TOTL],Women who believe a husband is justified in beating his wife when she refuses sex with him (%) [SG.VAW.REFU.ZS]
0,2013,YR2013,Afghanistan,AFG,NaN,NaN,10014.577,NaN,NaN,62.494,50.968831,31731688.0,75.627,652860.0,NaN,7733964.0,NaN
1,2013,YR2013,Albania,ALB,NaN,10957.0,5064.127,0.00000,0.0,77.702,15.635917,2895092.0,44.613,28750.0,NaN,1603505.0,NaN
2,2013,YR2013,Algeria,DZA,NaN,NaN,134465.223,99.44899,0.0,75.418,31.009854,38338562.0,30.424,2381740.0,NaN,26674438.0,NaN
3,2013,YR2013,American Samoa,ASM,NaN,NaN,NaN,NaN,NaN,NaN,3.457885,55307.0,12.652,200.0,NaN,48310.0,NaN
4,2013,YR2013,Andorra,AND,NaN,NaN,476.710,NaN,NaN,NaN,10.942684,80788.0,11.463,470.0,NaN,71527.0,NaN


## Data cleaning

### Column cleaning

In [4]:
df.count() # Compte les non NaN par colonne

Time                                                                                                               217
Time Code                                                                                                          217
Country Name                                                                                                       217
Country Code                                                                                                       217
Annual freshwater withdrawals, total (billion cubic meters) [ER.H2O.FWTL.K3]                                        16
Children out of school, primary [SE.PRM.UNER]                                                                      125
CO2 emissions (kt) [EN.ATM.CO2E.KT]                                                                                206
Electricity production from oil, gas and coal sources (% of total) [EG.ELC.FOSL.ZS]                                142
Electricity production from renewable sources, e

##### Les colonnes suivantes sont à supprimer (trop de NaN) : 
##### Annual freshwater, Total greenhouse gas emission, Women who believe...

In [16]:
filtered_col_df = df.drop(['Annual freshwater withdrawals, total (billion cubic meters) [ER.H2O.FWTL.K3]'],axis=1)
filtered_col_df = filtered_col_df.drop(['Total greenhouse gas emissions (kt of CO2 equivalent) [EN.ATM.GHGT.KT.CE]'],axis=1)
filtered_col_df = filtered_col_df.drop(['Women who believe a husband is justified in beating his wife when she refuses sex with him (%) [SG.VAW.REFU.ZS]'],axis=1)

filtered_col_df.head()

,Time,Time Code,Country Name,Country Code,"Children out of school, primary [SE.PRM.UNER]",CO2 emissions (kt) [EN.ATM.CO2E.KT],"Electricity production from oil, gas and coal sources (% of total) [EG.ELC.FOSL.ZS]","Electricity production from renewable sources, excluding hydroelectric (% of total) [EG.ELC.RNWX.ZS]","Life expectancy at birth, total (years) [SP.DYN.LE00.IN]","PM2.5 air pollution, mean annual exposure (micrograms per cubic meter) [EN.ATM.PM25.MC.M3]","Population, total [SP.POP.TOTL]",Rural population (% of total population) [SP.RUR.TOTL.ZS],Surface area (sq. km) [AG.SRF.TOTL.K2],Urban population [SP.URB.TOTL]
0,2013,YR2013,Afghanistan,AFG,NaN,10014.577,NaN,NaN,62.494,50.968831,31731688.0,75.627,652860.0,7733964.0
1,2013,YR2013,Albania,ALB,10957.0,5064.127,0.00000,0.0,77.702,15.635917,2895092.0,44.613,28750.0,1603505.0
2,2013,YR2013,Algeria,DZA,NaN,134465.223,99.44899,0.0,75.418,31.009854,38338562.0,30.424,2381740.0,26674438.0
3,2013,YR2013,American Samoa,ASM,NaN,NaN,NaN,NaN,NaN,3.457885,55307.0,12.652,200.0,48310.0
4,2013,YR2013,Andorra,AND,NaN,476.710,NaN,NaN,NaN,10.942684,80788.0,11.463,470.0,71527.0


In [19]:
filtered_col_df.isna().sum(axis=0)

Time                                                                                                     0
Time Code                                                                                                0
Country Name                                                                                             0
Country Code                                                                                             0
Children out of school, primary [SE.PRM.UNER]                                                           92
CO2 emissions (kt) [EN.ATM.CO2E.KT]                                                                     11
Electricity production from oil, gas and coal sources (% of total) [EG.ELC.FOSL.ZS]                     75
Electricity production from renewable sources, excluding hydroelectric (% of total) [EG.ELC.RNWX.ZS]    76
Life expectancy at birth, total (years) [SP.DYN.LE00.IN]                                                18
PM2.5 air pollution, mean annual expo

### Rows cleaning

In [20]:
filtered_df = filtered_col_df.dropna() # On enlève toutes les lignes qui contiennent des NA

filtered_df.tail()

,Time,Time Code,Country Name,Country Code,"Children out of school, primary [SE.PRM.UNER]",CO2 emissions (kt) [EN.ATM.CO2E.KT],"Electricity production from oil, gas and coal sources (% of total) [EG.ELC.FOSL.ZS]","Electricity production from renewable sources, excluding hydroelectric (% of total) [EG.ELC.RNWX.ZS]","Life expectancy at birth, total (years) [SP.DYN.LE00.IN]","PM2.5 air pollution, mean annual exposure (micrograms per cubic meter) [EN.ATM.PM25.MC.M3]","Population, total [SP.POP.TOTL]",Rural population (% of total population) [SP.RUR.TOTL.ZS],Surface area (sq. km) [AG.SRF.TOTL.K2],Urban population [SP.URB.TOTL]
210,2013,YR2013,"Venezuela, RB",VEN,244940.0,183922.052,32.165476,0.000000,74.035,24.765075,30317848.0,11.867,912050.0,26720029.0
211,2013,YR2013,Vietnam,VNM,127071.0,147230.050,58.337998,0.115950,75.663,27.815889,91497725.0,67.571,330972.0,29671797.0
214,2013,YR2013,"Yemen, Rep.",YEM,599564.0,25496.651,100.000000,0.000000,64.291,59.399450,25576322.0,66.441,527970.0,8583158.0
215,2013,YR2013,Zambia,ZMB,363385.0,3956.693,0.142857,0.000000,59.982,27.650733,15153210.0,59.135,752610.0,6192359.0
216,2013,YR2013,Zimbabwe,ZWE,398579.0,11675.728,46.060606,1.734587,58.053,23.448376,15054506.0,67.346,390760.0,4915898.0


In [22]:
filtered_df.isna().sum(axis=0)

Time                                                                                                    0
Time Code                                                                                               0
Country Name                                                                                            0
Country Code                                                                                            0
Children out of school, primary [SE.PRM.UNER]                                                           0
CO2 emissions (kt) [EN.ATM.CO2E.KT]                                                                     0
Electricity production from oil, gas and coal sources (% of total) [EG.ELC.FOSL.ZS]                     0
Electricity production from renewable sources, excluding hydroelectric (% of total) [EG.ELC.RNWX.ZS]    0
Life expectancy at birth, total (years) [SP.DYN.LE00.IN]                                                0
PM2.5 air pollution, mean annual exposure (mic

In [23]:
filtered_df.count(axis=0)
# Il reste 96 pays dans le df

Time                                                                                                    96
Time Code                                                                                               96
Country Name                                                                                            96
Country Code                                                                                            96
Children out of school, primary [SE.PRM.UNER]                                                           96
CO2 emissions (kt) [EN.ATM.CO2E.KT]                                                                     96
Electricity production from oil, gas and coal sources (% of total) [EG.ELC.FOSL.ZS]                     96
Electricity production from renewable sources, excluding hydroelectric (% of total) [EG.ELC.RNWX.ZS]    96
Life expectancy at birth, total (years) [SP.DYN.LE00.IN]                                                96
PM2.5 air pollution, mean annual expo

## K-means

### Vision d'

## DBSCAN